In [2]:
# Setup environment
import os
import glob
import subprocess
import sys
import time
from scipy.io import netcdf as nc

import matplotlib.pyplot as plt
from os.path import expanduser

# Define output path
home = expanduser("~")
output_dir = os.path.join(home,'scratch')  # main output directory.
os.makedirs(os.path.dirname(output_dir), exist_ok=True)
print(output_dir)

/Users/sserbin/scratch


In [3]:
# Grab latest CTSM docker container from Docker hub
source = "serbinsh/ctsm_containers"
docker_container_tag = "ctsm-fates_next_api-fates_sci.1.23.0_api.7.1.0"
container_name = ''.join([source,":",docker_container_tag])
print(container_name)
docker_pull_command = ''.join(["docker pull ",source,":",docker_container_tag])
os.system(docker_pull_command)

# you should see docker pull progress in the Jupyter shell

serbinsh/ctsm_containers:ctsm-fates_next_api-fates_sci.1.23.0_api.7.1.0


0

In [ ]:
# Define run parameters for CTSM
start_year = '1990-01-01'  # you need to place this in the single quotations
num_years = 5
met_start = 1990
met_end = 1996
run_type = "startup"
hostname = "modex"
user = "clmuser"

# Define input/output paths (host:container)
cesm_input_data_paths = "~/Data/cesm_input_data:/data"
output_path = ''.join([output_dir,":/ctsm_output"])
print(output_path)

# Define script to run
ctsm_case_build_script = "/ctsm_run_scripts/create_case_ctsmfates_1pt_example_1x1brazil.sh"


